# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import matplotlib.pyplot as plt


In [2]:
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)


Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())



## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size,image_index,y =100,z = 100):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = image_index #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = image[5:-5,5:-5,:]
                    new_w = z
                    new_h = y
                    image_resized = imresize(image,[new_h,new_w,3])
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_seq = len(folder_list)%batch_size
        batch_data = np.zeros((remaining_seq,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_seq,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_seq): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*remaining_seq)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (num_batches*remaining_seq)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                image = image[5:-5,5:-5,:]
                new_w = z
                new_h = y
                image_resized = imresize(image,[new_h,new_w,3])

                batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255#normalise and feed in the image
                batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255#normalise and feed in the image
                batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255#normalise and feed in the image

            batch_labels[folder, int(t[folder + (num_batches*remaining_seq)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

### Plotting

In [7]:
curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30    # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Final Model using Conv3D 
- epoch  = 30
- batch_size = 100   
- frames = 24, removing some first and last images 
- image size 100 * 100
- dropout = 0.5
- dense neuron  = 128



In [19]:
batch_size = 100    #experiment with the batch size
image_index = [i for i in range(4,28)]

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation , Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

dropout = 0.5
dense_neurons = 128

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same',
         input_shape=(len(image_index),100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(dense_neurons,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(dense_neurons,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(5,activation='softmax'))



Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [21]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 24, 100, 100, 16)  1312      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 24, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 12, 50, 50, 16)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 12, 50, 50, 32)    4128      
_________________________________________________________________
activation_6 (Activation)    (None, 12, 50, 50, 32)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 12, 50, 50, 32)    128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [22]:
train_generator = generator(train_path, train_doc, batch_size,image_index)
val_generator = generator(val_path, val_doc, batch_size,image_index)

In [23]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [24]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [25]:
history16 = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  valSource path =  train ; batch size = 100
 ; batch size = 100
Epoch 1/30
7/7 [==============================] - 75s 11s/step - loss: 2.0877 - categorical_accuracy: 0.2519 - val_loss: 2.0019 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2021-02-0518_23_59.653761/model-00001-2.13711-0.25792-2.00188-0.41000.h5
Epoch 2/30
7/7 [==============================] - 11s 2s/step - loss: 0.7374 - categorical_accuracy: 0.2200 - val_loss: 1.2570 - val_categorical_accuracy: 0.4700

Epoch 00002: saving model to model_init_2021-02-0518_23_59.653761/model-00002-0.73744-0.21995-1.25696-0.47000.h5
Epoch 3/30
7/7 [==============================] - 11s 2s/step - loss: 0.4745 - categorical_accuracy: 0.2948 - val_loss: 1.6086 - val_categorical_accuracy: 0.3800

Epoch 00003: saving model to model_init_2021-02-0518_23_59.653761/model-00003-0.47451-0.29478-1.60858-0.38000.h5
Epoch 4/30
7/7 [==============================] - 11s 2s/step - loss: 0.4866 - categorical_ac